# Data preprocessing

In [1]:
#이 코드 하나로 단일모델, 분리모델 모두 전처리 가능합니다.
#이 부분을 채워주셔야 합니다.
DATA_DIR = 'cropped_v2.1' # 혹은 data 등. 아래의 DATA_PATH를 참고하여 기입
SEPERA = True # 분리하는 모델(종현모델)을 사용할 경우 True
NORM = False #분리하지 않는 모델(정재님의 모델)을 사용할 경우 True
# SEPERA와 NORM은 둘다 킬 수도 있습니다.(하지만 둘 중 하나는 반드시 켜야 합니다.)

## Gender Label Correction

In [2]:
DATA_PATH = f'/opt/ml/input/{DATA_DIR}/train/'
FILE_NAME = 'train.csv'

In [3]:
import pandas as pd
#해당 경로에 train.csv 파일이 있어야 정상작동합니다.
df = pd.read_csv(DATA_PATH+FILE_NAME)

In [4]:
# 잘못 분류된 데이터 폴더명을 기입
gender_f_to_m = [
    '006359', 
    '006360', 
    '006361', 
    '006362', 
    '006363', 
    '006364', 
]

gender_m_to_f = [
    '001498-1',
    '004432'
]

In [5]:
df_gender = df.copy()
df_gender.set_index(['id'], inplace=True)
# df_gender

In [6]:
for i in gender_f_to_m:
    df_gender.at[i, 'gender'] = 'male'

for i in gender_m_to_f:
    df_gender.at[i, 'gender'] = 'female'

In [7]:
df_new = df.copy()
df_new['gender'] = df_gender['gender'].tolist()
# df_new
# df_new.gender.value_counts()

In [8]:
df_new.to_csv(f'/opt/ml/input/cropped_v2.1/train/train_label.csv', index=False)

#### Mask label은 수정됐다고 가정

## Data Classify

In [8]:
df_label = df_new.copy()
#df_label.gender.value_counts()

In [9]:
#새로운 데이터프레임 생성
name = ['incorrect_mask','mask1','mask2','mask3','mask4','mask5','normal']
mask = ['Incorrect', 'Wear', 'Wear', 'Wear', 'Wear', 'Wear', 'NotWear']
dfff = pd.DataFrame()

for i in range(7):
    dff = df_label
    dff['img_path'] = df['path'] + '/' + name[i] + '.jpg'
    dff['mask'] = mask[i]
    dfff = dfff.append(dff, ignore_index=True).reset_index(drop=True)
# dfff

In [10]:
# 일반 모델을 사용할 경우
if NORM == True : 
    # 예측대상인 타깃 변수 생성
    dfff['target'] = 0
    dfff['target'] = ((dfff['mask']=='NotWear')*12 + 
                      (dfff['mask']=='Incorrect')*6 +
                      (dfff['gender']=='female')*3 + 
                      (dfff['age']>=30)*1 + 
                      (dfff['age']>=58)*1)
    # 기존 train.csv와 같은 위치에 새로운 csv 생성
    dfff.to_csv(DATA_PATH+'train_classified.csv', index=False)
    dfff.to_csv(DATA_PATH+'train_cls.csv', index=False)

In [11]:
# 분리모델을 사용할 경우
if SEPERA == True:
    # 예측대상인 타깃 변수 생성 mask
    dfff['target'] = 0
    dfff['target'] = ((dfff['mask']=='NotWear')*12 + 
                      (dfff['mask']=='Incorrect')*6)
    # 기존 train.csv와 같은 위치에 새로운 csv 생성
    dfff.to_csv(DATA_PATH+'train_classified_mask.csv', index=False)

    # 예측대상인 타깃 변수 생성 gender
    dfff['target'] = 0
    dfff['target'] = ((dfff['gender']=='female')*3)
    # 기존 train.csv와 같은 위치에 새로운 csv 생성
    dfff.to_csv(DATA_PATH+'train_classified_gender.csv', index=False)

    # 예측대상인 타깃 변수 생성 age
    dfff['target'] = 0
    dfff['target'] = ((dfff['age']>=30)*1 + 
                      (dfff['age']>=58)*1)
    # 기존 train.csv와 같은 위치에 새로운 csv 생성
    dfff.to_csv(DATA_PATH+'train_classified_age.csv', index=False)

## Folder Classification

In [12]:
import os
import shutil
import pandas as pd

In [13]:
def folder_classification(data_dir, target):
    # data_dir 은 데이터 폴더 이름 cropped_v2나 data 등
    DATA_PATH = f'/opt/ml/input/{data_dir}/train/'
    IMG_PATH = f'/opt/ml/input/{data_dir}/train/images/'
    if target == 'all':
        NEW_IMG_PATH = f'/opt/ml/input/{data_dir}/train/images_classified/'
        train_classified = 'train_classified.csv'
    else : 
        TRAIN_IMG_PATH = f'/opt/ml/input/{data_dir}/train/train_{target}/'
        #VALID_IMG_PATH = f'/opt/ml/input/{data_dir}/train/valid_{target}/'
        train_classified = f'train_classified_{target}.csv' 
    
    df_cls = pd.read_csv(DATA_PATH+train_classified)
    df_cls.sort_values(by=['id'], inplace=True)
    df_cls.reset_index(inplace=True)
    df_cls.drop(['index'], axis=1, inplace=True)
    
    #df_cls.to_csv(DATA_PATH+'test.csv', index=False)
    
    # 새로 분류해서 저장할 디렉토리 생성
    if not os.path.isdir(TRAIN_IMG_PATH):
        os.makedirs(TRAIN_IMG_PATH)
    
    '''
    if not os.path.isdir(VALID_IMG_PATH):
        os.makedirs(VALID_IMG_PATH)
    '''

    # 타깃 클래스를 이름으로 갖는 디렉토리들을 생성
    for i in df_cls['target'].unique():
        if not os.path.isdir(TRAIN_IMG_PATH+str(i)):
            os.makedirs(TRAIN_IMG_PATH+str(i))
    
    '''
    for i in df_cls['target'].unique():
        if not os.path.isdir(VALID_IMG_PATH+str(i)):
            os.makedirs(VALID_IMG_PATH+str(i))\
    '''

    # images 폴더 내의 개별 이미지 들의 경로를 저장
    #path_list = sorted([p for p in os.listdir(IMG_PATH) if '._' not in p])
    target_list = df_cls['target'].unique()

    # 만들어둔 새로운 경로에 각각의 이미지를 타깃별로 분류해서 복사 붙여넣기.
    cnt = 0
    for t in target_list:
        idx_list = df_cls.index[df_cls['target'] == t].tolist()
        
        split_idx = int(len(idx_list) * 0.8)
        train_idx = idx_list[:split_idx]
        #valid_idx = idx_list[split_idx:]
        
        for i in train_idx:
            label = str(df_cls.iloc[i, -1])
            img_path = df_cls.iloc[i, -3]
            path, file_name = img_path.split('/')
            shutil.copy(IMG_PATH+img_path, TRAIN_IMG_PATH+label+'/'+path + '_' + file_name)
            cnt += 1
        '''    
        for i in valid_idx:
        
            label = str(df_cls.iloc[i, -1])
            img_path = df_cls.iloc[i, -3]
            path, file_name = img_path.split('/')
        
            shutil.copy(IMG_PATH+img_path, VALID_IMG_PATH+label+'/'+path + '_' + file_name)
        
            cnt += 1 
        '''

In [14]:
if NORM == True : 
    folder_classification(DATA_DIR,'all')
if SEPERA == True :
    folder_classification(DATA_DIR,'mask')
    folder_classification(DATA_DIR,'gender')
    folder_classification(DATA_DIR,'age')